## S&P 600 Galaxy

Use this utlity to update the returns and std_dev fields within investment-options.csv

Globals

In [28]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = False
throttle_limit=1000
wait_time=30

In [29]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

In [31]:
# Read in sp600.csv
sp600 = pd.read_csv('sp600.csv')
sp600.drop(columns=['SEC filings', 'CIK'], inplace=True)
sp600.rename(columns={'Company':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp600.set_index("Symbol", inplace=True)
sp600

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AAP,"Advance Auto Parts, Inc.",Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina"
AAT,American Assets Trust,Real Estate,Diversified REITs,"San Diego, California"
ABCB,Ameris Bancorp,Financials,Regional Banks,"Atlanta, Georgia"
ABG,Asbury Automotive Group,Consumer Discretionary,Automotive Retail,"Duluth, Georgia"
ABM,"ABM Industries, Inc.",Industrials,Environmental & Facilities Services,"New York City, New York"
...,...,...,...,...
XRX,Xerox,Information Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"
YELP,"Yelp, Inc.",Communication Services,Interactive Media & Services,"San Francisco, California"
YOU,"Clear Secure, Inc.",Information Technology,Application Software,"New York City, New York"


In [32]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [33]:
# Map sp600 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp600['Asset Class'] = sp600.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp600.index = sp600.index.str.replace('.', '-', regex=False)
sp600

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AAP,"Advance Auto Parts, Inc.",US Stocks:Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina"
AAT,American Assets Trust,US Stocks:Real Estate,Diversified REITs,"San Diego, California"
ABCB,Ameris Bancorp,US Stocks:Financials,Regional Banks,"Atlanta, Georgia"
ABG,Asbury Automotive Group,US Stocks:Consumer Discretionary,Automotive Retail,"Duluth, Georgia"
ABM,"ABM Industries, Inc.",US Stocks:Industrials,Environmental & Facilities Services,"New York City, New York"
...,...,...,...,...
XRX,Xerox,US Stocks:Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"
YELP,"Yelp, Inc.",US Stocks:Communication Services,Interactive Media & Services,"San Francisco, California"
YOU,"Clear Secure, Inc.",US Stocks:Technology,Application Software,"New York City, New York"


In [34]:
# Drop invalid symbols.
sp600.drop(['BSIG', 'DRQ', 'JBT', 'SGH', 'UCBI', 'VVI'], inplace=True)


In [35]:
# Make symbols list.
symbols = list(sp600.index)

In [36]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries, throttle_limit=throttle_limit, wait_time=wait_time)
bb.compile_timeseries(symbols)

AAP AAT ABCB ABG ABM ABR ACA ACAD ACIW ACLS ACT ADEA ADMA ADNT ADUS AEIS AEO AESI AGO AGYS AHCO AHH AIN AIR AKR AL ALEX ALG ALGT ALKS ALRM AMN AMPH AMR AMSF AMTM AMWD ANDE ANGI ANIP AORT AOSL APAM APLE APOG ARCB ARI ARLO AROC ARR ARWR ASIX ASO ASTE ASTH ATEN ATGE AUB AVA AVAV AVNS AWI AWR AX AXL AZTA AZZ BANC BANF BANR BCC BCPC BDN BFH BFS BGC BGS BHE BHLB BJRI BKE BKU BL BLFS BLMN BMI BOH BOOT BOX BRC BRKL BTU BWA BXMT CABO CAKE CAL CALM CALX CARG CARS CASH CATY CBRL CBU CC CCOI CCS CE CENT CENTA CENX CERT CEVA CFFN CHCO CHEF CLB CLSK CNK CNMD CNR CNS CNXN COHU COLL CON COOP CORT CPF CPK CPRX CRC CRGY CRI CRK CRSR CRVL CSGS CSR CSWI CTKB CTRE CTS CUBI CURB CVBF CVCO CVI CWEN CWEN-A CWK CWT CXM CXW DAN DCOM DEA DEI DFH DFIN DGII DIOD DLX DNOW DOCN DORM DRH DV DVAX DXC DXPE DY EAT ECG ECPG EFC EGBN EIG ELME EMBC ENOV ENR ENVA EPAC EPC EPRT ESE ESI ETD ETSY EVTC EXPI EXTR EYE EZPW FBK FBNC FBP FBRT FCF FCPT FDP FELE FFBC FHB FIZZ FL FMC FORM FOXF FRPT FSS FTDR FTRE FUL FULT FUN FWRD GBX 

In [37]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,AAP,AAT,ABCB,ABG,ABM,ABR,ACA,ACAD,ACIW,ACLS,...,WT,WWW,XHR,XNCR,XPEL,XRX,YELP,YOU,ZD,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,141.14,30.21,29.11,68.12,27.67,5.35,27.57,16.65,27.14,17.79,...,5.93,28.10,14.18,35.79,NaN,14.14,34.57,NaN,59.27,22.70
2019-01-03,145.54,30.22,29.01,67.84,27.38,5.43,27.59,16.17,25.16,16.57,...,5.60,27.53,14.10,34.86,NaN,13.83,33.64,NaN,57.62,21.39
2019-01-04,141.93,30.46,30.14,68.47,28.32,5.58,28.30,17.57,26.17,17.61,...,5.75,28.12,14.62,36.58,NaN,14.36,34.56,NaN,60.02,22.24
2019-01-07,144.02,30.94,31.34,70.66,29.62,5.72,29.71,18.23,26.51,17.85,...,5.81,28.27,14.62,38.09,NaN,14.67,34.83,NaN,60.60,22.38
2019-01-08,143.02,31.58,31.26,69.83,30.95,5.81,29.48,18.54,27.39,17.78,...,5.96,28.69,15.11,38.01,NaN,15.16,35.51,NaN,60.88,22.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,32.63,19.01,56.96,224.45,48.51,11.51,80.17,14.66,52.78,48.60,...,8.71,12.98,10.73,10.60,27.26,4.39,35.58,25.06,30.85,33.71
2025-04-29,33.31,18.52,59.00,216.26,48.75,11.58,80.12,14.43,53.52,47.87,...,8.77,13.11,10.62,10.58,28.56,4.44,35.80,25.31,30.75,34.17
2025-04-30,32.72,18.73,58.60,218.14,48.74,11.53,80.07,14.60,53.36,48.98,...,8.70,13.05,10.68,11.02,28.64,4.41,35.08,24.68,29.53,33.96


In [38]:
# Calculate Annual Returns.
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

AAP    -53.10
AAT     -5.39
ABCB    28.94
ABG      5.75
ABM     15.94
        ...  
XRX    -59.85
YELP   -12.57
YOU     48.17
ZD     -37.22
ZWS     12.91
Length: 596, dtype: float64

In [39]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [40]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [41]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [42]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [43]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [44]:
# Calculate 1 year, 3 year, and 5 year annualized standard deviation.
std_dev_1yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=1)
std_dev_3yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev_5yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=5)

In [45]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]

In [46]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp600.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd_1yr = std_dev_1yr[symbol]*100
    sd_3yr = std_dev_3yr[symbol]*100
    sd_5yr = std_dev_5yr[symbol]*100

    out.append((
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}",'
        '"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
    ).format(
        symbol, description, asset_class, ret_1mo, ret_3mo, ret_1yr, ret_3yr,
        ret_5yr, _vola, _ds_vola, sd_1yr, sd_3yr, sd_5yr
    ))

In [47]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')